In [1]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import pandas as pd
import pymongo
from pymongo import MongoClient
import tkinter as tk
from tkinter import *
import numpy as np

'''
import the three tables into python
this code appears to have a lot repeating and so would seem a candidate for a function, but there's a lot of vairety, there
is only one repeating identical line in the 4, therefore I decided to write seperately 
''' 

def load_csv_mongo():
    client = MongoClient()
    db=client.food_hs

    inv = db.inventory
    df = pd.read_csv("Inventroy.csv")
    records_ = df.to_dict(orient = 'records')
    result = db.inv.insert_many(records_ )

    ins = db.inspections
    df = pd.read_csv("Inspections.csv")
    records_ = df.to_dict(orient = 'records')
    result = db.ins.insert_many(records_ )

    vio = db.violations
    df = pd.read_csv("violations.csv")
    records_ = df.to_dict(orient = 'records')
    result = db.vio.insert_many(records_ )

def parse_clean():
    global inspect
    global invent
    global violat
    client = MongoClient()
    db = client['food_hs']
    inspect = pd.DataFrame(list(db.ins.find({})))
    invent = pd.DataFrame(list(db.inv.find({})))
    violat = pd.DataFrame(list(db.vio.find({})))
    
    #remove any rows which have only 'na' values. This is both for now and for when users add data later
    inspect.dropna(how='all')
    invent.dropna(how='all')
    violat.dropna(how='all')

    #replace # with space in inventroy and inspections in set columns, as some columns have # as part of the legitamate string
    invent['OWNER ADDRESS']=invent['OWNER ADDRESS'].str.replace('#',' ')
    invent['OWNER CITY']=invent['OWNER CITY'].str.replace('#',' ')
    invent['FACILITY CITY']=invent['FACILITY CITY'].str.replace('#',' ')
    invent['FACILITY ADDRESS']=invent['FACILITY ADDRESS'].str.replace('#',' ')
    
    inspect['FACILITY CITY']=inspect['FACILITY CITY'].str.replace('#',' ')
    inspect['FACILITY ADDRESS']=inspect['FACILITY ADDRESS'].str.replace('#',' ')

    #in 4 columns in inverntroy pandas has replaced ' ' with the placeholder &#160; (#is removed above) so I am replacing 
    #these with space. I am only doing this for these columns as this is the only place it occurs, and so will save 
    #searching 26 fields, just the 4 necessary. Inspections has the same in 2 columns
    invent['OWNER ADDRESS']=invent['OWNER ADDRESS'].str.replace('& 160;',' ')
    invent['OWNER CITY']=invent['OWNER CITY'].str.replace('& 160;',' ')
    invent['FACILITY CITY']=invent['FACILITY CITY'].str.replace('& 160;',' ')
    invent['FACILITY ADDRESS']=invent['FACILITY ADDRESS'].str.replace('& 160;',' ')
    
    inspect['FACILITY CITY']=inspect['FACILITY CITY'].str.replace('& 160;',' ')
    inspect['FACILITY ADDRESS']=inspect['FACILITY ADDRESS'].str.replace('& 160;',' ')

    #remove all inactives from inspections
    inspect = inspect[inspect['PROGRAM STATUS']!= 'INACTIVE']

    # Convert eg 08/23/2018 from string to datetime this allows for the analysis later
    inspect['ACTIVITY DATE'] =  pd.to_datetime(inspect['ACTIVITY DATE'], format='%m/%d/%Y')

    #make new column for the risk values
    inspect['RISK LEVEL']=inspect['PE DESCRIPTION'].values

    #only retain the risk level string in RISK LEVEL column
    inspect['RISK LEVEL'] = inspect['RISK LEVEL'].str.split('SEATS ').str[1].str.strip()

    #remove all categories of risk from PE DESCRIPTION

    inspect['PE DESCRIPTION'] = inspect['PE DESCRIPTION'].str.replace('MODERATE RISK' , '')
    inspect['PE DESCRIPTION'] = inspect['PE DESCRIPTION'].str.replace('HIGH RISK' , '')
    inspect['PE DESCRIPTION'] = inspect['PE DESCRIPTION'].str.replace('LOW RISK' , '')
    
    #remove dupicates from all datasets , this is to check for future problems
    violat=violat.drop_duplicates(subset=None, keep='first', inplace=False)
    inspect=inspect.drop_duplicates(subset=None, keep='first', inplace=False)
    invent=invent.drop_duplicates(subset=None, keep='first', inplace=False)

    #delete dirty data and parse clean datasets to Mongo
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["food_hs"]
    mycol1 = mydb["inv"]
    mycol2 = mydb["ins"]
    mycol3 = mydb["vio"]

    mycol1.drop()
    mycol2.drop()
    mycol3.drop()

    client = MongoClient()
    db=client.food_hs

    inv = db.inventory
    records_ = invent.to_dict(orient = 'records')
    result = db.inv.insert_many(records_ )

    ins = db.inventory
    records_ = inspect.to_dict(orient = 'records')
    result = db.ins.insert_many(records_ )

    vio = db.inventory
    records_ = violat.to_dict(orient = 'records')
    result = db.vio.insert_many(records_ )

def violations_score():    
    #new df merged from vio and ins for use to see score and number of violations by establishment
    num_df = pd.merge(violat, inspect, on = 'SERIAL NUMBER')

    #new column which holds the amount of vioaltions an esablishment has been guilt of
    num_df['VIOLATIONS'] = num_df.groupby('FACILITY ID')['FACILITY ID'].transform('count')

    #sort the num_df by date
    num_df = pd.DataFrame(num_df.sort_values(by='ACTIVITY DATE'))
    
    num_df = num_df[['SERIAL NUMBER','FACILITY ID','FACILITY NAME','SCORE','VIOLATIONS']]
    num_df = pd.DataFrame(num_df.drop_duplicates('FACILITY ID', keep = 'last'))

    #make result df just shows the two relevant columns
    num_df = num_df[['SCORE','VIOLATIONS']]

    #sort the values on violations ready for graphing
    num_df.sort_values('VIOLATIONS', axis = 0, ascending = True, 
                     inplace = True, na_position ='last') 
    
    #select and store all records with more than 300 violations
    indexNames = num_df[num_df['VIOLATIONS'] > 200].index

    # Delete these row indexes from dataFrame
    num_df.drop(indexNames , inplace=True)
    
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)

    x = num_df.SCORE
    y = num_df.VIOLATIONS
    
    plt.scatter(x, y)
    plt.title('Violations and current score')
    plt.xlabel('Score')
    plt.ylabel('Violations')
    plt.savefig('score_violations.jpg', bbox_inches='tight') 
    plt.show() 
    plt.close()
        
    window = Tk()
    im = Image.open('score_violations.jpg')
    ph = ImageTk.PhotoImage(im)
    label = Label(window, image=ph)
    label.image=ph
    tk.Label(root, image =label).pack
    tk.Button(root, text="Return to averages menu", command=lambda: root.destroy()).pack()

    
def facility_count_vio():
    #new df merged from vio and ins for use to see score and number of violations by establishment
    num_df = pd.merge(violat, inspect, on = 'SERIAL NUMBER')

    #new column which holds the amount of vioaltions an esablishment has been guilt of
    num_df['VIOLATIONS'] = num_df.groupby('FACILITY ID')['FACILITY ID'].transform('count')

    #sort the num_df by date
    num_df = pd.DataFrame(num_df.sort_values(by='ACTIVITY DATE'))
    
    num_df = num_df[['SERIAL NUMBER','FACILITY ID','FACILITY NAME','SCORE','VIOLATIONS']]
    num_df = pd.DataFrame(num_df.drop_duplicates('FACILITY ID', keep = 'last'))

    #sort the values on violations ready for graphing
    num_df.sort_values('VIOLATIONS', axis = 0, ascending = True, 
                     inplace = True, na_position ='last') 
    
    #select and store all records with more than 300 violations
    indexNames = num_df[num_df['VIOLATIONS'] > 200].index

    x = num_df.VIOLATIONS
    count = num_df['VIOLATIONS'].value_counts() 
    plt.bar(count, x)
    plt.show() 
    plt.savefig('score_facility.png', bbox_inches='tight')    
    plt.close()
    
def average_all():
    #create new dataframes which contain each years records 
    ins17 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2017-01-01','2017-12-31'))]
    ins18 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2018-01-01','2018-12-31'))]
    ins19 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2019-01-01','2019-12-31'))]
    global averages_all
    averages_all = ("Mean for 2017: ",ins17["SCORE"].mean(),"\n","Median for 2017: ",ins17["SCORE"].median(),"\n", 
                    "Mode for 2017: ",ins17["SCORE"].mode()
                    
                    ,"\n\n","Mean for 2018: ",ins18["SCORE"].mean(),"\n","Median for 2018: ",ins18["SCORE"].median(),
                    "\n""Mode for 2018: ",ins18["SCORE"].mode()
                    
                    ,"\n\n","Mean for 2019: ",ins19["SCORE"].mean(),"\n","Median for 2019: ",ins19["SCORE"].median(),
                    "\n","Mode for 2019: ",ins19["SCORE"].mode())
    
def averages(city,year):
    def calculate():
        city_df = merge_df.loc[merge_df['FACILITY CITY_x']==city]
        mean_city = city_df["SCORE"].mean()
        mode_city = city_df["SCORE"].mode()
        median_city = city_df["SCORE"].median()
        global averages
        
        mean_text = (city," mean ",year,":", mean_city)
        median_text = (city," median ",year,":",median_city)
        mode_text = (city," mode ",year,":",mode_city) 
        averages = (mean_text,"\n",median_text,"\n",mode_text,"\n")
        
    if year == '2017':
        ins17 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2017-01-01','2017-12-31'))]
        merge_df = pd.merge(invent, ins17, how='inner', on = 'RECORD ID')
        calculate()
    elif year == '2018':
        ins18 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2018-01-01','2018-12-31'))]
        merge_df = pd.merge(invent, ins18, how='inner', on = 'RECORD ID')
        calculate()    
    elif year == '2019':
        ins19 = inspect[inspect['ACTIVITY DATE'].isin(pd.date_range('2019-01-01','2019-12-31'))]
        merge_df = pd.merge(invent, ins19, how='inner', on = 'RECORD ID')
        calculate()

class GUI_app(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self._frame = None
        self.switch_frame(StartPage)

    def switch_frame(self, frame_class):
        """Destroys current frame and replaces it with a new one."""
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()

def City_Av():
    root = Tk()
    options = inspect['FACILITY CITY'].unique().tolist()     

    root.title("Please select city and year")
    root.geometry("400x200")

    var_city = StringVar(root)
    var_city.set(options[0]) # default value
    var_year = StringVar(root) 
    var_year.set('2017')
    dd1 = OptionMenu(root, var_city, *options).pack()
    dd2 = OptionMenu(root, var_year, '2017','2018','2019').pack()
    def label():
            tk.Label(root,text = averages).pack()
    
    def run_averages():
        location = var_city.get()
        date = var_year.get()
        averages(location,date)
        label()

    tk.Button(root, text="Calculate averages", command=lambda: run_averages()).pack()
    tk.Button(root, text="Return to averages menu", command=lambda: root.destroy()).pack()

def Av_all():
    root = Tk()
    average_all()
    tk.Label(root, text=averages_all).pack(side="top", fill="x", pady=10)
    tk.Button(root, text="Return to averages menu", command=lambda: root.destroy()).pack()

class StartPage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Label(self, text="Welcome").pack(side="top", fill="x", pady=10)
        master.title("Main Menu")
        master.geometry("350x190")
        tk.Button(self, text="Transfer csv to mongoDB", command = lambda: load_csv_mongo()).pack()        
        tk.Button(self, text="Import and clean data ready to use", command = lambda: parse_clean()).pack()
        tk.Button(self, text="Averages", command=lambda: master.switch_frame(Averages)).pack()
        tk.Button(self, text="Visualise correlation between score and violations", command=lambda: violations_score()).pack()
        tk.Button(self, text="Visualise establishment and number of violations", command=lambda: facility_count_vio()).pack()
        
class Averages(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        tk.Label(self, text="Welcome to the averages page").pack(side="top", fill="x", pady=10)
        tk.Button(self, text="Averages for all", command=lambda: Av_all()).pack()
        tk.Button(self, text="Averages for cities", command=lambda: City_Av()).pack()
        tk.Button(self, text="Return to start page",
                  command=lambda: master.switch_frame(StartPage)).pack()
 
if __name__ == "__main__":
    app = GUI_app()
    app.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\bensk\AppData\Local\Continuum\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-177ce2266e50>", line 288, in <lambda>
    tk.Button(self, text="Import and clean data ready to use", command = lambda: parse_clean()).pack()
  File "<ipython-input-1-177ce2266e50>", line 94, in parse_clean
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
NameError: name 'pymongo' is not defined
